In [1]:
import pandas as pd #used for importing
import math as m 
import numpy as np #used for matrix manipulation
from scipy.interpolate import interp2d #used for interpolating Perple_X outputs
from scipy.interpolate import interp1d #used for interpolating evenly spaced P-T conditions from modeling
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt #plotting package
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap #used for defining user-created colormap
import matplotlib.colors as col

In [2]:
##USER INPUTS------------------------------------------------------------------------------------------------------------
##-----------------------------------------------------------------------------------------------------------------------
##for P-T conditions used in Perplex Calculation. P-T range and nodes used in werami
Tst=273
Tend=1700
Pst=1500
Pend=80000
nodes=250

##thickness of layers in the downgoing section. should sum to 10100. The code extracts P-T data every 100m from slabtop
##to 10km below, with each P-T path controlling devolatilization for the underlying 100m, hence the 10km P-T path dictates
##devolatilization in the depth range of 10 to 10.1km
lay_thk=[2100,5000,1400,300,300,0] #last value is for sediment thickness, which is changed later for each loop

##creates arrays for setting up interpolation, Tmin to tmaxx with an increment matching that used in werami
Tinc=(Tend-Tst)/(nodes-1)
Pinc=(Pend-Pst)/(nodes-1)
Tppx=np.arange(Tst,Tend+Tinc,Tinc)
Pppx=np.arange(Pst,Pend+Pinc,Pinc)
##----------------------------------------------------------------------------------------------------------------------
##----------------------------------------------------------------------------------------------------------------------

#This section reads in the Holt & Condit dataset and normalizes all ages steps to the same size, from xmin:xmax=500
#this works for all time steps from 1 to 118, but can be defined by user to omit early steps (as done here)

#create 4 dimensional indexing array
#D1=surface, 5km, 10km
#D2=X,Y,P_lith,P_tot,T
#D3=idx_len=# of time steps considered, here we omit the first 7, beginning with t=5.6Myr, i.e. 8:118=111
#D4=nromalized Xdistance=; inc=round((xmax-xmin-1)/500,3); X_new=np.arange(xmin,xmax,inc)==500
t_start=8 #time step to begin with, here it is the 8th step, corresponding to t=5.6Myr
idx_len=107 #max ages step included in input file
data_mat_norm=np.zeros((3,5,idx_len,500))

#define naming format
file_slabtop='holt4e20/{}.slabtop.txt'
file_5km='holt4e20/{}.5km.txt'
file_10km='holt4e20/{}.10km.txt'

#read in X,Y,P_lith,P_tot,T for each time step
for i in range(t_start,idx_len+t_start): #t_start here signifies the starting t step
    PT_st=pd.read_csv(file_slabtop.format(i), header=None, delim_whitespace=True)
    PT_5k=pd.read_csv(file_5km.format(i), header=None, delim_whitespace=True)
    PT_10k=pd.read_csv(file_10km.format(i), header=None, delim_whitespace=True)

    #convert to numpy arrays
    PT_st=PT_st.to_numpy()
    PT_5k=PT_5k.to_numpy()
    PT_10k=PT_10k.to_numpy()

    #flips arrays to be ordered by increasing X
    PT_st=np.flipud(PT_st)
    PT_5k=np.flipud(PT_5k)
    PT_10k=np.flipud(PT_10k)
    
    #determines length of each file for normalization
    mxst=PT_st.shape[0]-1
    mx5k=PT_5k.shape[0]-1
    mx10k=PT_10k.shape[0]-1

#-----------------------------------------------------------------------------------------------------------------
    xmin=m.ceil(PT_st[1,0]) #determines minimum X value at slabtop
            
    lmax_pos=0 # sets position of max X value for next loop
    
    #this loop iterates through the X data to determine if and where the X value starts appreciably decreasing. 
    #this step is required for interpolation as 1dinterp requires a monotonically increasing independent array
    #the cutoff is set to be where the indexed X-value is greater than the next highest index X-value by a value of 5 
    #value of 5 is somewhat arbitrary.
    
    #IF THIS BLOCK PRESENTS AN ERROR: print(i) AND INSPECT THE FILE TO SEE HOW TO CHANGE THE LOGIC. NOT THE BEST FIX
    #BUT NEEDED DUE TO THE NATURE OF EXTRACTING CONTINUOUS DATA FROM FINITE ELEMENTS.
    
    for j in range(50,mxst):
        if (((PT_st[j,0]>PT_st[j+1,0]) and (PT_st[j,0]-PT_st[j+1,0]>5)) or (PT_st[j,0]-PT_st[j+1,0]<-9)):
            lmax_pos=j
            break  
            
    #if there is no point where the previous loop is satisfied the index for the max X-value is set to the max index 
    #of the array        
    if lmax_pos==0: 
        lmax_pos=mxst
    
    #determines the maximum X value at the previously determined index, and creates the X-range for 1dinterpolation
    xmax=m.floor(PT_st[lmax_pos,0])
    inc=round((xmax-xmin-1)/500,3)
    X_new=np.arange(xmin,xmax,inc)

    #In a few instances the X-range is longer than we want, so we resize by deleting the last few elements.
    if X_new.shape[0]>500:
        X_new=X_new[0:500]
    
    #creates the interpolation functions for Y, P_lith, P_tot, and T as a function of X
    F_Yst=interp1d(PT_st[1:lmax_pos,0],PT_st[1:lmax_pos,1])
    F_Plst=interp1d(PT_st[1:lmax_pos,0],PT_st[1:lmax_pos,2])
    F_Ptst=interp1d(PT_st[1:lmax_pos,0],PT_st[1:lmax_pos,3])
    F_Tst=interp1d(PT_st[1:lmax_pos,0],PT_st[1:lmax_pos,4])
    
    #determines the new values for Y, P_lith, P_tot, and T as a function of the new X-range
    Y_new=F_Yst(X_new)
    Pl_new=F_Plst(X_new)
    Pt_new=F_Ptst(X_new)
    T_new=F_Tst(X_new)
    
    #adds new values of X, Y, P_lith, P_tot, and T to the 4d array. The above section has only computed new values for
    #the slabtop. The next two sections repeat the process for PT conditions at depths of 5 and 10km below the slabtop
    #notice that the indexing shows i-1, this is because we have omitted the first time step from the matrix, it has a bunch
    #of weird artifacts.
    data_mat_norm[0,0,i-8,:]=X_new
    data_mat_norm[0,1,i-8,:]=Y_new
    data_mat_norm[0,2,i-8,:]=Pl_new
    data_mat_norm[0,3,i-8,:]=Pt_new
    data_mat_norm[0,4,i-8,:]=T_new
#----------------------------------------------------------------------------------------------------------------
#5km interpolation    
    xmin=m.ceil(PT_5k[1,0])
            
    lmax_pos=0
    
    for j in range(50,mx5k):
        if (((PT_5k[j,0]>PT_5k[j+1,0]) and (PT_5k[j,0]-PT_5k[j+1,0]>5)) or (PT_5k[j,0]-PT_5k[j+1,0]<-9)):
            lmax_pos=j
            break  
            
    if lmax_pos==0:
        lmax_pos=mx5k
        
    xmax=m.floor(PT_5k[lmax_pos,0])
    inc=round((xmax-xmin-1)/500,3)
    X_new=np.arange(xmin,xmax,inc)

    if X_new.shape[0]>500:
        X_new=X_new[0:500]
    #X_new=X_new[0:490]
    
    F_Y5k=interp1d(PT_5k[1:lmax_pos,0],PT_5k[1:lmax_pos,1])
    F_Pl5k=interp1d(PT_5k[1:lmax_pos,0],PT_5k[1:lmax_pos,2])
    F_Pt5k=interp1d(PT_5k[1:lmax_pos,0],PT_5k[1:lmax_pos,3])
    F_T5k=interp1d(PT_5k[1:lmax_pos,0],PT_5k[1:lmax_pos,4])
    
    Y_new=F_Y5k(X_new)
    Pl_new=F_Pl5k(X_new)
    Pt_new=F_Pt5k(X_new)
    T_new=F_T5k(X_new)
    
    data_mat_norm[1,0,i-8,:]=X_new
    data_mat_norm[1,1,i-8,:]=Y_new
    data_mat_norm[1,2,i-8,:]=Pl_new
    data_mat_norm[1,3,i-8,:]=Pt_new
    data_mat_norm[1,4,i-8,:]=T_new
#----------------------------------------------------------------------------------------------------------------
#10km interpolation
    xmin=m.ceil(PT_10k[1,0])
            
    lmax_pos=0
    
    for j in range(50,mx10k):
        if (((PT_10k[j,0]>PT_10k[j+1,0]) and (PT_10k[j,0]-PT_10k[j+1,0]>5)) or (PT_10k[j,0]-PT_10k[j+1,0]<-9)):
            lmax_pos=j
            break  
            
    if lmax_pos==0:
        lmax_pos=mx10k
        
    xmax=m.floor(PT_10k[lmax_pos,0])
    inc=round((xmax-xmin-1)/500,3)
    X_new=np.arange(xmin,xmax,inc)

    if X_new.shape[0]>500:
        X_new=X_new[0:500]
    #X_new=X_new[0:490]
    
    F_Y10k=interp1d(PT_10k[1:lmax_pos,0],PT_10k[1:lmax_pos,1])
    F_Pl10k=interp1d(PT_10k[1:lmax_pos,0],PT_10k[1:lmax_pos,2])
    F_Pt10k=interp1d(PT_10k[1:lmax_pos,0],PT_10k[1:lmax_pos,3])
    F_T10k=interp1d(PT_10k[1:lmax_pos,0],PT_10k[1:lmax_pos,4])
    
    Y_new=F_Y10k(X_new)
    Pl_new=F_Pl10k(X_new)
    Pt_new=F_Pt10k(X_new)
    T_new=F_T10k(X_new)
    
    data_mat_norm[2,0,i-8,:]=X_new
    data_mat_norm[2,1,i-8,:]=Y_new
    data_mat_norm[2,2,i-8,:]=Pl_new
    data_mat_norm[2,3,i-8,:]=Pt_new
    data_mat_norm[2,4,i-8,:]=T_new

In [3]:
abmin=m.ceil(np.min(data_mat_norm[:,0,:,0])) #absolute minimum of all X-values
abmax=m.floor(np.max(data_mat_norm[:,0,:,499])) #absolute maximum of all X-values
even_inc=abmax-abmin
xrange=np.arange(abmin,abmax) #indexing array used to find start and stop locations for filling D4 of array

##loads in auxillary file which contains ages associated with the time steps. only loads age column
age=pd.read_csv('holt4e20/age_convergence.txt', header=None, usecols=[1], delim_whitespace=True)
age=age.to_numpy()
##creates 2 array of ages for use in contour plotting
age=np.repeat(age[8:idx_len+8],500,axis=1)

convergence=pd.read_csv('holt4e20/age_convergence.txt', header=None, usecols=[2], delim_whitespace=True)
convergence=convergence.to_numpy()
##creates 2 array of ages for use in contour plotting
convergence=np.repeat(convergence[8:idx_len+8],even_inc,axis=1)

In [4]:
##This section creates a new 4d array and fills it with even spaced X,Y,P_tot,T data.
##The array is first poulated with NaNs, its maximum dimension encompasses the entire X-range of all P-T paths in 
##1km increments (here 1300). For any given slab the array is populated only over a subset range, with the remaining array
##elements retaining NaN.
## The final results in a 4d array with: 
##D1=X,Y,P_tot,T
##D2=in-slab data, from the interface to 10km depth with a spacing of 100m
##D3=each individual time step from 1 to 118 (0 to 117 as index)
##D4=km spacing over X-range, I have used indexing to fill the final matrix with data interpolated in the loops.

array2=np.empty((4,101,idx_len,even_inc)) #creating of hypercube array
array2[:]=np.nan #filling of array elements with NaN

for k in range(0,idx_len): #ARE WE SURE THIS SHOULD ONLY GO TO idx_len???
    
    section=data_mat_norm[:,:,k,:]
    min2=m.ceil(np.max(section[:,0,0]))
    max2=m.floor(np.min(section[:,0,499]))
    range2=np.arange(min2,max2)
    
    start=np.where(xrange==min2)[0][0]
    stop=np.where(xrange==max2)[0][0]
    
    FYst=interp1d(section[0,0,:],section[0,1,:])
    FY5k=interp1d(section[1,0,:],section[1,1,:])
    FY10k=interp1d(section[2,0,:],section[2,1,:])
    
    FPst=interp1d(section[0,0,:],section[0,3,:])
    FP5k=interp1d(section[1,0,:],section[1,3,:])
    FP10k=interp1d(section[2,0,:],section[2,3,:])
    
    FTst=interp1d(section[0,0,:],section[0,4,:])
    FT5k=interp1d(section[1,0,:],section[1,4,:])
    FT10k=interp1d(section[2,0,:],section[2,4,:])
    
    Yst=FYst(range2)
    Y5k=FY5k(range2)
    Y10k=FY10k(range2)

    Pst=FPst(range2)
    P5k=FP5k(range2)
    P10k=FP10k(range2)

    Tst=FTst(range2)
    T5k=FT5k(range2)
    T10k=FT10k(range2)
    
    X_incremental=range2.T
    X_incremental=X_incremental.reshape(X_incremental.shape[0],-1)
    X_incremental=np.repeat(X_incremental,101,axis=1)
    array2[0,:,k,start:stop]=X_incremental.T
    
    for l in range(0,101):
        if l<51:
            array2[1,l,k,start:stop]=((1-l/50)*Yst+l/50*Y5k)
            array2[2,l,k,start:stop]=((1-l/50)*Pst+l/50*P5k)
            array2[3,l,k,start:stop]=((1-l/50)*Tst+l/50*T5k)
        else:
            array2[1,l,k,start:stop]=((1-(l-50)/50)*Y5k+(l-50)/50*Y10k)
            array2[2,l,k,start:stop]=((1-(l-50)/50)*P5k+(l-50)/50*P10k)
            array2[3,l,k,start:stop]=((1-(l-50)/50)*T5k+(l-50)/50*T10k)

In [5]:
##read in the werami output files for the layers of interest. In this simple case, we have used 5km of MORB over
##5.1km of serpentinite. The werami output needs to be edited to remove all header info except the final column headers.
##for this simple case the files contain info on rock densisty, fluid density, bound fluid, melt, and melt density
##the first 2 columns of the werami files (corresponding to T and P) are not read in (usecols keyword).
##delim_whitespace is always required for reading in werami files because the spacing between columns is non-uniform.

#Below is latest and greatest perple_x outputs
lay0a=pd.read_csv('holt2e20/terrigenous.tab', usecols=[2,3,4,5,6], delim_whitespace=True)
lay0b=pd.read_csv('holt2e20/pelagic.tab', usecols=[2,3,4,5,6], delim_whitespace=True)
lay0c=pd.read_csv('holt2e20/carbonate.tab', usecols=[2,3,4,5,6], delim_whitespace=True)
lay0d=pd.read_csv('holt2e20/turbidite.tab', usecols=[2,3,4,5,6], delim_whitespace=True)
lay0e=pd.read_csv('holt2e20/diatomite.tab', usecols=[2,3,4,5,6], delim_whitespace=True)
lay1=pd.read_csv('holt2e20/ue90.tab', usecols=[2,3,4,5,6], delim_whitespace=True)
lay2=pd.read_csv('holt2e20/le90.tab', usecols=[2,3,4,5,6], delim_whitespace=True)
lay3=pd.read_csv('holt2e20/sd90.tab', usecols=[2,3,4,5,6], delim_whitespace=True)
lay4=pd.read_csv('holt2e20/g90.tab', usecols=[2,3,4,5,6], delim_whitespace=True)
lay5=pd.read_csv('holt2e20/hart_peridotite_new.tab', usecols=[2,3,4,5,6], delim_whitespace=True)

##convert files to numpy arrays
#sediment types
lay0a=lay0a.to_numpy() #terrigenous
lay0b=lay0b.to_numpy() #pelagic
lay0c=lay0c.to_numpy() #carbonate
lay0d=lay0d.to_numpy() #turbidite
lay0e=lay0e.to_numpy() #diatomite

#all other compositions
lay1=lay1.to_numpy()
lay2=lay2.to_numpy()
lay3=lay3.to_numpy()
lay4=lay4.to_numpy()
lay5=lay5.to_numpy()

##repeat the above processes for the other layer/layers read in from Perple_X
lay0a_den=lay0a[:,0]
lay0a_aqden=lay0a[:,1]
lay0a_fl=lay0a[:,2]
lay0a_melt=lay0a[:,3]
lay0a_mden=lay0a[:,4]

lay0a_den=lay0a_den.reshape(nodes,nodes)
lay0a_aqden=lay0a_aqden.reshape(nodes,nodes)
lay0a_fl=lay0a_fl.reshape(nodes,nodes)
lay0a_melt=lay0a_melt.reshape(nodes,nodes)
lay0a_mden=lay0a_mden.reshape(nodes,nodes)

lay0a_den=np.nan_to_num(lay0a_den,nan=0)
lay0a_aqden=np.nan_to_num(lay0a_aqden,nan=0)
lay0a_fl=np.nan_to_num(lay0a_fl,nan=0)
lay0a_melt=np.nan_to_num(lay0a_melt,nan=0)
lay0a_mden=np.nan_to_num(lay0a_mden,nan=0)

F_den_0a=interp2d(Tppx,Pppx,lay0a_den)
F_aqden_0a=interp2d(Tppx,Pppx,lay0a_aqden)
F_fl_0a=interp2d(Tppx,Pppx,lay0a_fl)
F_melt_0a=interp2d(Tppx,Pppx,lay0a_melt)
F_mden_0a=interp2d(Tppx,Pppx,lay0a_mden)

##repeat the above processes for the other layer/layers read in from Perple_X
lay0b_den=lay0b[:,0]
lay0b_aqden=lay0b[:,1]
lay0b_fl=lay0b[:,2]
lay0b_melt=lay0b[:,3]
lay0b_mden=lay0b[:,4]

lay0b_den=lay0b_den.reshape(nodes,nodes)
lay0b_aqden=lay0b_aqden.reshape(nodes,nodes)
lay0b_fl=lay0b_fl.reshape(nodes,nodes)
lay0b_melt=lay0b_melt.reshape(nodes,nodes)
lay0b_mden=lay0b_mden.reshape(nodes,nodes)

lay0b_den=np.nan_to_num(lay0b_den,nan=0)
lay0b_aqden=np.nan_to_num(lay0b_aqden,nan=0)
lay0b_fl=np.nan_to_num(lay0b_fl,nan=0)
lay0b_melt=np.nan_to_num(lay0b_melt,nan=0)
lay0b_mden=np.nan_to_num(lay0b_mden,nan=0)

F_den_0b=interp2d(Tppx,Pppx,lay0b_den)
F_aqden_0b=interp2d(Tppx,Pppx,lay0b_aqden)
F_fl_0b=interp2d(Tppx,Pppx,lay0b_fl)
F_melt_0b=interp2d(Tppx,Pppx,lay0b_melt)
F_mden_0b=interp2d(Tppx,Pppx,lay0b_mden)

##repeat the above processes for the other layer/layers read in from Perple_X
lay0c_den=lay0c[:,0]
lay0c_aqden=lay0c[:,1]
lay0c_fl=lay0c[:,2]
lay0c_melt=lay0c[:,3]
lay0c_mden=lay0c[:,4]

lay0c_den=lay0c_den.reshape(nodes,nodes)
lay0c_aqden=lay0c_aqden.reshape(nodes,nodes)
lay0c_fl=lay0c_fl.reshape(nodes,nodes)
lay0c_melt=lay0c_melt.reshape(nodes,nodes)
lay0c_mden=lay0c_mden.reshape(nodes,nodes)

lay0c_den=np.nan_to_num(lay0c_den,nan=0)
lay0c_aqden=np.nan_to_num(lay0c_aqden,nan=0)
lay0c_fl=np.nan_to_num(lay0c_fl,nan=0)
lay0c_melt=np.nan_to_num(lay0c_melt,nan=0)
lay0c_mden=np.nan_to_num(lay0c_mden,nan=0)

F_den_0c=interp2d(Tppx,Pppx,lay0c_den)
F_aqden_0c=interp2d(Tppx,Pppx,lay0c_aqden)
F_fl_0c=interp2d(Tppx,Pppx,lay0c_fl)
F_melt_0c=interp2d(Tppx,Pppx,lay0c_melt)
F_mden_0c=interp2d(Tppx,Pppx,lay0c_mden)

##repeat the above processes for the other layer/layers read in from Perple_X
lay0d_den=lay0d[:,0]
lay0d_aqden=lay0d[:,1]
lay0d_fl=lay0d[:,2]
lay0d_melt=lay0d[:,3]
lay0d_mden=lay0d[:,4]

lay0d_den=lay0d_den.reshape(nodes,nodes)
lay0d_aqden=lay0d_aqden.reshape(nodes,nodes)
lay0d_fl=lay0d_fl.reshape(nodes,nodes)
lay0d_melt=lay0d_melt.reshape(nodes,nodes)
lay0d_mden=lay0d_mden.reshape(nodes,nodes)

lay0d_den=np.nan_to_num(lay0d_den,nan=0)
lay0d_aqden=np.nan_to_num(lay0d_aqden,nan=0)
lay0d_fl=np.nan_to_num(lay0d_fl,nan=0)
lay0d_melt=np.nan_to_num(lay0d_melt,nan=0)
lay0d_mden=np.nan_to_num(lay0d_mden,nan=0)

F_den_0d=interp2d(Tppx,Pppx,lay0d_den)
F_aqden_0d=interp2d(Tppx,Pppx,lay0d_aqden)
F_fl_0d=interp2d(Tppx,Pppx,lay0d_fl)
F_melt_0d=interp2d(Tppx,Pppx,lay0d_melt)
F_mden_0d=interp2d(Tppx,Pppx,lay0d_mden)

##repeat the above processes for the other layer/layers read in from Perple_X
lay0e_den=lay0e[:,0]
lay0e_aqden=lay0e[:,1]
lay0e_fl=lay0e[:,2]
lay0e_melt=lay0e[:,3]
lay0e_mden=lay0e[:,4]

lay0e_den=lay0e_den.reshape(nodes,nodes)
lay0e_aqden=lay0e_aqden.reshape(nodes,nodes)
lay0e_fl=lay0e_fl.reshape(nodes,nodes)
lay0e_melt=lay0e_melt.reshape(nodes,nodes)
lay0e_mden=lay0e_mden.reshape(nodes,nodes)

lay0e_den=np.nan_to_num(lay0e_den,nan=0)
lay0e_aqden=np.nan_to_num(lay0e_aqden,nan=0)
lay0e_fl=np.nan_to_num(lay0e_fl,nan=0)
lay0e_melt=np.nan_to_num(lay0e_melt,nan=0)
lay0e_mden=np.nan_to_num(lay0e_mden,nan=0)

F_den_0e=interp2d(Tppx,Pppx,lay0e_den)
F_aqden_0e=interp2d(Tppx,Pppx,lay0e_aqden)
F_fl_0e=interp2d(Tppx,Pppx,lay0e_fl)
F_melt_0e=interp2d(Tppx,Pppx,lay0e_melt)
F_mden_0e=interp2d(Tppx,Pppx,lay0e_mden)

##split file into individual properties
lay1_den=lay1[:,0]
lay1_aqden=lay1[:,1]
lay1_fl=lay1[:,2]
lay1_melt=lay1[:,3]
lay1_mden=lay1[:,4]


##reshape individual files onto square grid [node,node]=[250,250] here
lay1_den=lay1_den.reshape(nodes,nodes)
lay1_aqden=lay1_aqden.reshape(nodes,nodes)
lay1_fl=lay1_fl.reshape(nodes,nodes)
lay1_melt=lay1_melt.reshape(nodes,nodes)
lay1_mden=lay1_mden.reshape(nodes,nodes)

##replace NaN with 0. Could be done in perple_X itself as well, but here we do it in post.
lay1_den=np.nan_to_num(lay1_den,nan=0)
lay1_aqden=np.nan_to_num(lay1_aqden,nan=0)
lay1_fl=np.nan_to_num(lay1_fl,nan=0)
lay1_melt=np.nan_to_num(lay1_melt,nan=0)
lay1_mden=np.nan_to_num(lay1_mden,nan=0)

##Create the interpolation functions for density, fluid density, bound H2O, melt, and melt density
F_den_1=interp2d(Tppx,Pppx,lay1_den)
F_aqden_1=interp2d(Tppx,Pppx,lay1_aqden)
F_fl_1=interp2d(Tppx,Pppx,lay1_fl)
F_melt_1=interp2d(Tppx,Pppx,lay1_melt)
F_mden_1=interp2d(Tppx,Pppx,lay1_mden)

##repeat the above processes for the other layer/layers read in from Perple_X
lay2_den=lay2[:,0]
lay2_aqden=lay2[:,1]
lay2_fl=lay2[:,2]
lay2_melt=lay2[:,3]
lay2_mden=lay2[:,4]

lay2_den=lay2_den.reshape(nodes,nodes)
lay2_aqden=lay2_aqden.reshape(nodes,nodes)
lay2_fl=lay2_fl.reshape(nodes,nodes)
lay2_melt=lay2_melt.reshape(nodes,nodes)
lay2_mden=lay2_mden.reshape(nodes,nodes)

lay2_den=np.nan_to_num(lay2_den,nan=0)
lay2_aqden=np.nan_to_num(lay2_aqden,nan=0)
lay2_fl=np.nan_to_num(lay2_fl,nan=0)
lay2_melt=np.nan_to_num(lay2_melt,nan=0)
lay2_mden=np.nan_to_num(lay2_mden,nan=0)

F_den_2=interp2d(Tppx,Pppx,lay2_den)
F_aqden_2=interp2d(Tppx,Pppx,lay2_aqden)
F_fl_2=interp2d(Tppx,Pppx,lay2_fl)
F_melt_2=interp2d(Tppx,Pppx,lay2_melt)
F_mden_2=interp2d(Tppx,Pppx,lay2_mden)

##repeat the above processes for the other layer/layers read in from Perple_X
lay3_den=lay3[:,0]
lay3_aqden=lay3[:,1]
lay3_fl=lay3[:,2]
lay3_melt=lay3[:,3]
lay3_mden=lay3[:,4]

lay3_den=lay3_den.reshape(nodes,nodes)
lay3_aqden=lay3_aqden.reshape(nodes,nodes)
lay3_fl=lay3_fl.reshape(nodes,nodes)
lay3_melt=lay3_melt.reshape(nodes,nodes)
lay3_mden=lay3_mden.reshape(nodes,nodes)

lay3_den=np.nan_to_num(lay3_den,nan=0)
lay3_aqden=np.nan_to_num(lay3_aqden,nan=0)
lay3_fl=np.nan_to_num(lay3_fl,nan=0)
lay3_melt=np.nan_to_num(lay3_melt,nan=0)
lay3_mden=np.nan_to_num(lay3_mden,nan=0)

F_den_3=interp2d(Tppx,Pppx,lay3_den)
F_aqden_3=interp2d(Tppx,Pppx,lay3_aqden)
F_fl_3=interp2d(Tppx,Pppx,lay3_fl)
F_melt_3=interp2d(Tppx,Pppx,lay3_melt)
F_mden_3=interp2d(Tppx,Pppx,lay3_mden)

##repeat the above processes for the other layer/layers read in from Perple_X
lay4_den=lay4[:,0]
lay4_aqden=lay4[:,1]
lay4_fl=lay4[:,2]
lay4_melt=lay4[:,3]
lay4_mden=lay4[:,4]

lay4_den=lay4_den.reshape(nodes,nodes)
lay4_aqden=lay4_aqden.reshape(nodes,nodes)
lay4_fl=lay4_fl.reshape(nodes,nodes)
lay4_melt=lay4_melt.reshape(nodes,nodes)
lay4_mden=lay4_mden.reshape(nodes,nodes)

lay4_den=np.nan_to_num(lay4_den,nan=0)
lay4_aqden=np.nan_to_num(lay4_aqden,nan=0)
lay4_fl=np.nan_to_num(lay4_fl,nan=0)
lay4_melt=np.nan_to_num(lay4_melt,nan=0)
lay4_mden=np.nan_to_num(lay4_mden,nan=0)

F_den_4=interp2d(Tppx,Pppx,lay4_den)
F_aqden_4=interp2d(Tppx,Pppx,lay4_aqden)
F_fl_4=interp2d(Tppx,Pppx,lay4_fl)
F_melt_4=interp2d(Tppx,Pppx,lay4_melt)
F_mden_4=interp2d(Tppx,Pppx,lay4_mden)

##repeat the above processes for the other layer/layers read in from Perple_X
lay5_den=lay5[:,0]
lay5_aqden=lay5[:,1]
lay5_fl=lay5[:,2]
lay5_melt=lay5[:,3]
lay5_mden=lay5[:,4]

lay5_den=lay5_den.reshape(nodes,nodes)
lay5_aqden=lay5_aqden.reshape(nodes,nodes)
lay5_fl=lay5_fl.reshape(nodes,nodes)
lay5_melt=lay5_melt.reshape(nodes,nodes)
lay5_mden=lay5_mden.reshape(nodes,nodes)

lay5_den=np.nan_to_num(lay5_den,nan=0)
lay5_aqden=np.nan_to_num(lay5_aqden,nan=0)
lay5_fl=np.nan_to_num(lay5_fl,nan=0)
lay5_melt=np.nan_to_num(lay5_melt,nan=0)
lay5_mden=np.nan_to_num(lay5_mden,nan=0)

F_den_5=interp2d(Tppx,Pppx,lay5_den)
F_aqden_5=interp2d(Tppx,Pppx,lay5_aqden)
F_fl_5=interp2d(Tppx,Pppx,lay5_fl)
F_melt_5=interp2d(Tppx,Pppx,lay5_melt)
F_mden_5=interp2d(Tppx,Pppx,lay5_mden)

##create nan array to be filled later, this will contain all interpolated outputs, and can be used in tandem with array2 
##for plotting
wedges_brc=np.empty((56,107))
wedges_brc[:]=np.nan
wedges_ant=np.empty((56,107))
wedges_ant[:]=np.nan
wedges_700=np.empty((56,107))
wedges_700[:]=np.nan
wedges_chl=np.empty((56,107))
wedges_chl[:]=np.nan

In [6]:
h2o_capacity=pd.read_csv('Syracuse_Abers_margins.txt', delim_whitespace=True)
h2o_capacity=h2o_capacity.to_numpy()

In [7]:
wedge=pd.read_csv('wedge_PT/niu_perid_h2o_rho.tab', usecols=[2,3], delim_whitespace=True)
wedge=wedge.to_numpy()
wedge_fl=wedge[:,0]
wedge_fl=wedge_fl.reshape(nodes,nodes)
wedge_fl=np.nan_to_num(wedge_fl,nan=0)
F_fl_wedge=interp2d(Tppx,Pppx,wedge_fl)

wedge_out=np.empty((1,1,idx_len,even_inc))
wedge_out[:]=np.nan
for i in range(0,idx_len):
    xlow=np.where(xrange==np.nanmin(array2[0,:,i,:]))[0][0]
    xhigh=np.where(xrange==np.nanmax(array2[0,:,i,:]))[0][0]
    ##water content of serp at interface-----------------------------------------------------------------
    wedge_out[0,0,i,xlow:xhigh]=np.diag(F_fl_wedge(array2[3,0,i,xlow:xhigh]+273.15,array2[2,0,i,xlow:xhigh]*10000))

In [8]:
#perform analysis for every syracuse margin, progress is output by showing margin number from 0 to 55; takes a few minutes

for h in range(0,56):
    ppx_out=np.empty((7,101,idx_len,even_inc))
    ppx_out[:]=np.nan
    
    lay_thk[5]=h2o_capacity[h,1]

    thk_steps=int((sum(lay_thk)-100)/100)

    j1=int(lay_thk[0]/100)
    j2=int((lay_thk[0]+lay_thk[1])/100)
    j3=int((lay_thk[0]+lay_thk[1]+lay_thk[2])/100)
    j4=int((lay_thk[0]+lay_thk[1]+lay_thk[2]+lay_thk[3])/100)
    j5=int((lay_thk[0]+lay_thk[1]+lay_thk[2]+lay_thk[3]+lay_thk[4])/100)
    j6=int((lay_thk[0]+lay_thk[1]+lay_thk[2]+lay_thk[3]+lay_thk[4]+lay_thk[5])/100)
    
    for i in range(0,idx_len):
        xlow=np.where(xrange==np.nanmin(array2[0,:,i,:]))[0][0]
        xhigh=np.where(xrange==np.nanmax(array2[0,:,i,:]))[0][0]
        
        for j in range(0,j1):
            mm=thk_steps-j
            path_den=F_den_5(array2[3,mm,i,xlow:xhigh]+273.15, array2[2,mm,i,xlow:xhigh]*10000) #den of serp (no fluid)
            path_aqden=F_aqden_5(array2[3,mm,i,xlow:xhigh]+273.15, array2[2,mm,i,xlow:xhigh]*10000) #density of fluid
            path_fl=F_fl_5(array2[3,mm,i,xlow:xhigh]+273.15, array2[2,mm,i,xlow:xhigh]*10000) #bound H2O wt%
            path_melt=F_melt_5(array2[3,mm,i,xlow:xhigh]+273.15, array2[2,mm,i,xlow:xhigh]*10000) #melt wt% (no fluid)
            path_mden=F_mden_5(array2[3,mm,i,xlow:xhigh]+273.15, array2[2,mm,i,xlow:xhigh]*10000) #melt den (no fluid)
        
            ppx_out[0,mm,i,xlow:xhigh]=np.diag(path_den)#den of serp (no fluid)
            ppx_out[1,mm,i,xlow:xhigh]=np.diag(path_aqden)#density of fluid
            ppx_out[2,mm,i,xlow:xhigh]=np.diag(path_fl)#bound H2O wt%
            ppx_out[3,mm,i,xlow:xhigh]=np.diag(path_melt)#melt wt% (no fluid)
            ppx_out[4,mm,i,xlow:xhigh]=np.diag(path_mden)#melt den (no fluid)
            
            ##if melt present than fluid content is removed from ppx_out
            for n in range(xlow,xhigh+1):
                if ppx_out[3,mm,i,n]>0:
                    ppx_out[2,mm,i,n]=0
            ##determines the kg of bound H2O/m3 using density and bound H2O wt%
            ppx_out[5,mm,i,xlow:xhigh]=np.multiply(ppx_out[0,mm,i,xlow:xhigh],ppx_out[2,mm,i,xlow:xhigh])/100
            for o in range(xlow+1,xhigh+1):
                if ppx_out[5,mm,i,o]>ppx_out[5,mm,i,o-1]:
                    ppx_out[5,mm,i,o]=ppx_out[5,mm,i,o-1]
            ##determines incremental production of H2O in kg/m3        
            ppx_out[6,mm,i,xlow+1:xhigh]=np.subtract(ppx_out[5,mm,i,xlow+1:xhigh],ppx_out[5,mm,i,xlow:xhigh-1])*-1

        ##repeats the above for the upper layer (here MORB)
        for j in range(j1,j2):
            mm=thk_steps-j
            path_den=F_den_4(array2[3,mm,i,xlow:xhigh]+273.15, array2[2,mm,i,xlow:xhigh]*10000) #den of serp (no fluid)
            path_aqden=F_aqden_4(array2[3,mm,i,xlow:xhigh]+273.15, array2[2,mm,i,xlow:xhigh]*10000) #density of fluid
            path_fl=F_fl_4(array2[3,mm,i,xlow:xhigh]+273.15, array2[2,mm,i,xlow:xhigh]*10000) #bound H2O wt%
            path_melt=F_melt_4(array2[3,mm,i,xlow:xhigh]+273.15, array2[2,mm,i,xlow:xhigh]*10000) #melt wt% (no fluid)
            path_mden=F_mden_4(array2[3,mm,i,xlow:xhigh]+273.15, array2[2,mm,i,xlow:xhigh]*10000) #melt den (no fluid)
        
            ppx_out[0,mm,i,xlow:xhigh]=np.diag(path_den)#den of serp (no fluid)
            ppx_out[1,mm,i,xlow:xhigh]=np.diag(path_aqden)#density of fluid
            ppx_out[2,mm,i,xlow:xhigh]=np.diag(path_fl)#bound H2O wt%
            ppx_out[3,mm,i,xlow:xhigh]=np.diag(path_melt)#melt wt% (no fluid)
            ppx_out[4,mm,i,xlow:xhigh]=np.diag(path_mden)#melt den (no fluid)
        
            for n in range(xlow,xhigh+1):
                if ppx_out[3,mm,i,n]>0:
                    ppx_out[2,mm,i,n]=0

            ppx_out[5,mm,i,xlow:xhigh]=np.multiply(ppx_out[0,mm,i,xlow:xhigh],ppx_out[2,mm,i,xlow:xhigh])/100
            for o in range(xlow+1,xhigh+1):
                if ppx_out[5,mm,i,o]>ppx_out[5,mm,i,o-1]:
                    ppx_out[5,mm,i,o]=ppx_out[5,mm,i,o-1]
                
            ppx_out[6,mm,i,xlow+1:xhigh]=np.subtract(ppx_out[5,mm,i,xlow+1:xhigh],ppx_out[5,mm,i,xlow:xhigh-1])*-1
            
        for j in range(j2,j3):
            mm=thk_steps-j
            path_den=F_den_3(array2[3,mm,i,xlow:xhigh]+273.15, array2[2,mm,i,xlow:xhigh]*10000) #den of serp (no fluid)
            path_aqden=F_aqden_3(array2[3,mm,i,xlow:xhigh]+273.15, array2[2,mm,i,xlow:xhigh]*10000) #density of fluid
            path_fl=F_fl_3(array2[3,mm,i,xlow:xhigh]+273.15, array2[2,mm,i,xlow:xhigh]*10000) #bound H2O wt%
            path_melt=F_melt_3(array2[3,mm,i,xlow:xhigh]+273.15, array2[2,mm,i,xlow:xhigh]*10000) #melt wt% (no fluid)
            path_mden=F_mden_3(array2[3,mm,i,xlow:xhigh]+273.15, array2[2,mm,i,xlow:xhigh]*10000) #melt den (no fluid)
        
            ppx_out[0,mm,i,xlow:xhigh]=np.diag(path_den)#den of serp (no fluid)
            ppx_out[1,mm,i,xlow:xhigh]=np.diag(path_aqden)#density of fluid
            ppx_out[2,mm,i,xlow:xhigh]=np.diag(path_fl)#bound H2O wt%
            ppx_out[3,mm,i,xlow:xhigh]=np.diag(path_melt)#melt wt% (no fluid)
            ppx_out[4,mm,i,xlow:xhigh]=np.diag(path_mden)#melt den (no fluid)
        
            ##if melt present than fluid content is removed from ppx_out
            for n in range(xlow,xhigh+1):
                if ppx_out[3,mm,i,n]>0:
                    ppx_out[2,mm,i,n]=0
            ##determines the kg of bound H2O/m3 using density and bound H2O wt%
            ppx_out[5,mm,i,xlow:xhigh]=np.multiply(ppx_out[0,mm,i,xlow:xhigh],ppx_out[2,mm,i,xlow:xhigh])/100
            for o in range(xlow+1,xhigh+1):
                if ppx_out[5,mm,i,o]>ppx_out[5,mm,i,o-1]:
                    ppx_out[5,mm,i,o]=ppx_out[5,mm,i,o-1]
            ##determines incremental production of H2O in kg/m3        
            ppx_out[6,mm,i,xlow+1:xhigh]=np.subtract(ppx_out[5,mm,i,xlow+1:xhigh],ppx_out[5,mm,i,xlow:xhigh-1])*-1
            
        for j in range(j3,j4):
            mm=thk_steps-j
            path_den=F_den_2(array2[3,mm,i,xlow:xhigh]+273.15, array2[2,mm,i,xlow:xhigh]*10000) #den of serp (no fluid)
            path_aqden=F_aqden_2(array2[3,mm,i,xlow:xhigh]+273.15, array2[2,mm,i,xlow:xhigh]*10000) #density of fluid
            path_fl=F_fl_2(array2[3,mm,i,xlow:xhigh]+273.15, array2[2,mm,i,xlow:xhigh]*10000) #bound H2O wt%
            path_melt=F_melt_2(array2[3,mm,i,xlow:xhigh]+273.15, array2[2,mm,i,xlow:xhigh]*10000) #melt wt% (no fluid)
            path_mden=F_mden_2(array2[3,mm,i,xlow:xhigh]+273.15, array2[2,mm,i,xlow:xhigh]*10000) #melt den (no fluid)
        
            ppx_out[0,mm,i,xlow:xhigh]=np.diag(path_den)#den of serp (no fluid)
            ppx_out[1,mm,i,xlow:xhigh]=np.diag(path_aqden)#density of fluid
            ppx_out[2,mm,i,xlow:xhigh]=np.diag(path_fl)#bound H2O wt%
            ppx_out[3,mm,i,xlow:xhigh]=np.diag(path_melt)#melt wt% (no fluid)
            ppx_out[4,mm,i,xlow:xhigh]=np.diag(path_mden)#melt den (no fluid)
        
            ##if melt present than fluid content is removed from ppx_out
            for n in range(xlow,xhigh+1):
                if ppx_out[3,mm,i,n]>0:
                    ppx_out[2,mm,i,n]=0
            ##determines the kg of bound H2O/m3 using density and bound H2O wt%
            ppx_out[5,mm,i,xlow:xhigh]=np.multiply(ppx_out[0,mm,i,xlow:xhigh],ppx_out[2,mm,i,xlow:xhigh])/100
            for o in range(xlow+1,xhigh+1):
                if ppx_out[5,mm,i,o]>ppx_out[5,mm,i,o-1]:
                    ppx_out[5,mm,i,o]=ppx_out[5,mm,i,o-1]
            ##determines incremental production of H2O in kg/m3        
            ppx_out[6,mm,i,xlow+1:xhigh]=np.subtract(ppx_out[5,mm,i,xlow+1:xhigh],ppx_out[5,mm,i,xlow:xhigh-1])*-1
            
        for j in range(j4,j5):
            mm=thk_steps-j
            path_den=F_den_1(array2[3,mm,i,xlow:xhigh]+273.15, array2[2,mm,i,xlow:xhigh]*10000) #den of serp (no fluid)
            path_aqden=F_aqden_1(array2[3,mm,i,xlow:xhigh]+273.15, array2[2,mm,i,xlow:xhigh]*10000) #density of fluid
            path_fl=F_fl_1(array2[3,mm,i,xlow:xhigh]+273.15, array2[2,mm,i,xlow:xhigh]*10000) #bound H2O wt%
            path_melt=F_melt_1(array2[3,mm,i,xlow:xhigh]+273.15, array2[2,mm,i,xlow:xhigh]*10000) #melt wt% (no fluid)
            path_mden=F_mden_1(array2[3,mm,i,xlow:xhigh]+273.15, array2[2,mm,i,xlow:xhigh]*10000) #melt den (no fluid)
        
            ppx_out[0,mm,i,xlow:xhigh]=np.diag(path_den)#den of serp (no fluid)
            ppx_out[1,mm,i,xlow:xhigh]=np.diag(path_aqden)#density of fluid
            ppx_out[2,mm,i,xlow:xhigh]=np.diag(path_fl)#bound H2O wt%
            ppx_out[3,mm,i,xlow:xhigh]=np.diag(path_melt)#melt wt% (no fluid)
            ppx_out[4,mm,i,xlow:xhigh]=np.diag(path_mden)#melt den (no fluid)
        
            ##if melt present than fluid content is removed from ppx_out
            for n in range(xlow,xhigh+1):
                if ppx_out[3,mm,i,n]>0:
                    ppx_out[2,mm,i,n]=0
            ##determines the kg of bound H2O/m3 using density and bound H2O wt%
            ppx_out[5,mm,i,xlow:xhigh]=np.multiply(ppx_out[0,mm,i,xlow:xhigh],ppx_out[2,mm,i,xlow:xhigh])/100
            for o in range(xlow+1,xhigh+1):
                if ppx_out[5,mm,i,o]>ppx_out[5,mm,i,o-1]:
                    ppx_out[5,mm,i,o]=ppx_out[5,mm,i,o-1]
            ##determines incremental production of H2O in kg/m3        
            ppx_out[6,mm,i,xlow+1:xhigh]=np.subtract(ppx_out[5,mm,i,xlow+1:xhigh],ppx_out[5,mm,i,xlow:xhigh-1])*-1
            
        for j in range(j5,j6):
            mm=thk_steps-j
            if h2o_capacity[h,2]==1:
                path_den=F_den_0a(array2[3,mm,i,xlow:xhigh]+273.15, array2[2,mm,i,xlow:xhigh]*10000) #den of serp (no fluid)
                path_aqden=F_aqden_0a(array2[3,mm,i,xlow:xhigh]+273.15, array2[2,mm,i,xlow:xhigh]*10000) #density of fluid
                path_fl=F_fl_0a(array2[3,mm,i,xlow:xhigh]+273.15, array2[2,mm,i,xlow:xhigh]*10000) #bound H2O wt%
                path_melt=F_melt_0a(array2[3,mm,i,xlow:xhigh]+273.15, array2[2,mm,i,xlow:xhigh]*10000) #melt wt% (no fluid)
                path_mden=F_mden_0a(array2[3,mm,i,xlow:xhigh]+273.15, array2[2,mm,i,xlow:xhigh]*10000) #melt den (no fluid)
            elif h2o_capacity[h,2]==2:
                path_den=F_den_0b(array2[3,mm,i,xlow:xhigh]+273.15, array2[2,mm,i,xlow:xhigh]*10000) #den of serp (no fluid)
                path_aqden=F_aqden_0b(array2[3,mm,i,xlow:xhigh]+273.15, array2[2,mm,i,xlow:xhigh]*10000) #density of fluid
                path_fl=F_fl_0b(array2[3,mm,i,xlow:xhigh]+273.15, array2[2,mm,i,xlow:xhigh]*10000) #bound H2O wt%
                path_melt=F_melt_0b(array2[3,mm,i,xlow:xhigh]+273.15, array2[2,mm,i,xlow:xhigh]*10000) #melt wt% (no fluid)
                path_mden=F_mden_0b(array2[3,mm,i,xlow:xhigh]+273.15, array2[2,mm,i,xlow:xhigh]*10000) #melt den (no fluid)
            elif h2o_capacity[h,2]==3:
                path_den=F_den_0c(array2[3,mm,i,xlow:xhigh]+273.15, array2[2,mm,i,xlow:xhigh]*10000) #den of serp (no fluid)
                path_aqden=F_aqden_0c(array2[3,mm,i,xlow:xhigh]+273.15, array2[2,mm,i,xlow:xhigh]*10000) #density of fluid
                path_fl=F_fl_0c(array2[3,mm,i,xlow:xhigh]+273.15, array2[2,mm,i,xlow:xhigh]*10000) #bound H2O wt%
                path_melt=F_melt_0c(array2[3,mm,i,xlow:xhigh]+273.15, array2[2,mm,i,xlow:xhigh]*10000) #melt wt% (no fluid)
                path_mden=F_mden_0c(array2[3,mm,i,xlow:xhigh]+273.15, array2[2,mm,i,xlow:xhigh]*10000) #melt den (no fluid)
            elif h2o_capacity[h,2]==4:
                path_den=F_den_0d(array2[3,mm,i,xlow:xhigh]+273.15, array2[2,mm,i,xlow:xhigh]*10000) #den of serp (no fluid)
                path_aqden=F_aqden_0d(array2[3,mm,i,xlow:xhigh]+273.15, array2[2,mm,i,xlow:xhigh]*10000) #density of fluid
                path_fl=F_fl_0d(array2[3,mm,i,xlow:xhigh]+273.15, array2[2,mm,i,xlow:xhigh]*10000) #bound H2O wt%
                path_melt=F_melt_0d(array2[3,mm,i,xlow:xhigh]+273.15, array2[2,mm,i,xlow:xhigh]*10000) #melt wt% (no fluid)
                path_mden=F_mden_0d(array2[3,mm,i,xlow:xhigh]+273.15, array2[2,mm,i,xlow:xhigh]*10000) #melt den (no fluid)
            else:
                path_den=F_den_0e(array2[3,mm,i,xlow:xhigh]+273.15, array2[2,mm,i,xlow:xhigh]*10000) #den of serp (no fluid)
                path_aqden=F_aqden_0e(array2[3,mm,i,xlow:xhigh]+273.15, array2[2,mm,i,xlow:xhigh]*10000) #density of fluid
                path_fl=F_fl_0e(array2[3,mm,i,xlow:xhigh]+273.15, array2[2,mm,i,xlow:xhigh]*10000) #bound H2O wt%
                path_melt=F_melt_0e(array2[3,mm,i,xlow:xhigh]+273.15, array2[2,mm,i,xlow:xhigh]*10000) #melt wt% (no fluid)
                path_mden=F_mden_0e(array2[3,mm,i,xlow:xhigh]+273.15, array2[2,mm,i,xlow:xhigh]*10000) #melt den (no fluid)
                
            ppx_out[0,mm,i,xlow:xhigh]=np.diag(path_den)#den of serp (no fluid)
            ppx_out[1,mm,i,xlow:xhigh]=np.diag(path_aqden)#density of fluid
            ppx_out[2,mm,i,xlow:xhigh]=np.diag(path_fl)#bound H2O wt%
            ppx_out[3,mm,i,xlow:xhigh]=np.diag(path_melt)#melt wt% (no fluid)
            ppx_out[4,mm,i,xlow:xhigh]=np.diag(path_mden)#melt den (no fluid)
        
            ##if melt present than fluid content is removed from ppx_out
            for n in range(xlow,xhigh+1):
                if ppx_out[3,mm,i,n]>0:
                    ppx_out[2,mm,i,n]=0
            ##determines the kg of bound H2O/m3 using density and bound H2O wt%
            ppx_out[5,mm,i,xlow:xhigh]=np.multiply(ppx_out[0,mm,i,xlow:xhigh],ppx_out[2,mm,i,xlow:xhigh])/100
            for o in range(xlow+1,xhigh+1):
                if ppx_out[5,mm,i,o]>ppx_out[5,mm,i,o-1]:
                    ppx_out[5,mm,i,o]=ppx_out[5,mm,i,o-1]
            ##determines incremental production of H2O in kg/m3        
            ppx_out[6,mm,i,xlow+1:xhigh]=np.subtract(ppx_out[5,mm,i,xlow+1:xhigh],ppx_out[5,mm,i,xlow:xhigh-1])*-1
##-------------------------------------------------------------------------------------------------------------------------
##-------------------------------------------------------------------------------------------------------------------------
    dip_rad=np.empty((idx_len,even_inc))
    dip_rad[:]=np.nan
    for i in range(0,idx_len):
        xlow=np.where(xrange==np.nanmin(array2[0,:,i,:]))[0][0]
        xhigh=np.where(xrange==np.nanmax(array2[0,:,i,:]))[0][0]
        for j in range(xlow+1,xhigh+1):
            dip_rad[i,j]=m.atan(array2[1,0,i,j]-array2[1,0,i,j-1])
        dip_rad[i,xlow]=dip_rad[i,xlow+1]
    app_sr=np.multiply(np.cos(dip_rad),convergence)/100
    
    map_fl_mass=np.zeros((idx_len,even_inc))
    for i in range(0,idx_len):
        xlow=np.where(xrange==np.nanmin(array2[0,:,i,:]))[0][0]
        xhigh=np.where(xrange==np.nanmax(array2[0,:,i,:]))[0][0]
        t_i=np.subtract(array2[1,100,i,xlow:xhigh],array2[1,0,i,xlow:xhigh])*10
        fluid_col=np.sum(ppx_out[6,0:thk_steps+1,i,xlow:xhigh],axis=0)
        map_fl_mass[i,xlow:xhigh]=np.multiply(fluid_col,t_i)

    d700=np.zeros((idx_len,))
    dbrc=np.zeros((idx_len,))
    dant=np.zeros((idx_len,))
    dchl=np.zeros((idx_len,))
    for i in range(0,idx_len):
        d700[i,]=array2[1,0,i,np.where(array2[3,0,i,:]>=700)[0][0]]
        dbrc[i,]=array2[1,0,i,np.where(wedge_out[0,0,i,:]<=8)[0][0]]
        dant[i,]=array2[1,0,i,np.where(wedge_out[0,0,i,:]<=4)[0][0]]
        dchl[i,]=array2[1,0,i,np.where(wedge_out[0,0,i,:]<=1)[0][0]]
        
    forearc_flux_delt_brc=np.zeros((idx_len,even_inc))
    for i in range(0,idx_len):
        xlow=np.where(array2[1,0,i,:]>=30)[0][0]
        if np.nanmax(array2[1,0,i,:])>80:
            if dbrc[i]<=80:
                xhigh=np.where(array2[1,0,i,:]>=dbrc[i])[0][0]
            else:
                xhigh=np.where(array2[1,0,i,:]>=80)[0][0]
        else:
            xhigh=np.where(xrange==np.nanmax(array2[0,:,i,:]))[0][0] 
        forearc_flux_delt_brc[i,xlow:xhigh]=np.multiply(map_fl_mass[i,xlow:xhigh],app_sr[i,xlow:xhigh])

    forearc_flux_delt_ant=np.zeros((idx_len,even_inc))
    for i in range(0,idx_len):
        xlow=np.where(array2[1,0,i,:]>=30)[0][0]
        if np.nanmax(array2[1,0,i,:])>80:
            if dant[i]<=80:
                xhigh=np.where(array2[1,0,i,:]>=dant[i])[0][0]
            else:
                xhigh=np.where(array2[1,0,i,:]>=80)[0][0]
        else:
            xhigh=np.where(xrange==np.nanmax(array2[0,:,i,:]))[0][0] 
        forearc_flux_delt_ant[i,xlow:xhigh]=np.multiply(map_fl_mass[i,xlow:xhigh],app_sr[i,xlow:xhigh])

    forearc_flux_delt_700=np.zeros((idx_len,even_inc))
    for i in range(0,idx_len):
        xlow=np.where(array2[1,0,i,:]>=30)[0][0]
        if np.nanmax(array2[1,0,i,:])>80:
            if d700[i]<=80:
                xhigh=np.where(array2[1,0,i,:]>=d700[i])[0][0]
            else:
                xhigh=np.where(array2[1,0,i,:]>=80)[0][0]
        else:
            xhigh=np.where(xrange==np.nanmax(array2[0,:,i,:]))[0][0] 
        forearc_flux_delt_700[i,xlow:xhigh]=np.multiply(map_fl_mass[i,xlow:xhigh],app_sr[i,xlow:xhigh])

    forearc_flux_delt_chl=np.zeros((idx_len,even_inc))
    for i in range(0,idx_len):
        xlow=np.where(array2[1,0,i,:]>=30)[0][0]
        if np.nanmax(array2[1,0,i,:])>80:
            if dchl[i]<=80:
                xhigh=np.where(array2[1,0,i,:]>=dchl[i])[0][0]
            else:
                xhigh=np.where(array2[1,0,i,:]>=80)[0][0]
        else:
            xhigh=np.where(xrange==np.nanmax(array2[0,:,i,:]))[0][0] 
        forearc_flux_delt_chl[i,xlow:xhigh]=np.multiply(map_fl_mass[i,xlow:xhigh],app_sr[i,xlow:xhigh])
        
        
    forearc_delt_tot_brc=np.nansum(forearc_flux_delt_brc,axis=1)
    wedges_brc[h,:]=forearc_delt_tot_brc
    forearc_delt_tot_ant=np.nansum(forearc_flux_delt_ant,axis=1)
    wedges_ant[h,:]=forearc_delt_tot_ant
    forearc_delt_tot_700=np.nansum(forearc_flux_delt_700,axis=1)
    wedges_700[h,:]=forearc_delt_tot_700
    forearc_delt_tot_chl=np.nansum(forearc_flux_delt_chl,axis=1)
    wedges_chl[h,:]=forearc_delt_tot_chl
    print(h)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55


In [25]:
np.savetxt('holt4e20/wedges_brc_4e20.txt',wedges_brc)
np.savetxt('holt4e20/wedges_ant_4e20.txt',wedges_ant)
np.savetxt('holt4e20/wedges_700_4e20.txt',wedges_700)
np.savetxt('holt4e20/wedges_chl_4e20.txt',wedges_chl)